In [9]:
#Installing required packages

! pip install pyodbc
! pip install openpyxl
! pip install pandas

In [10]:
# Import date class from datetime module
from datetime import datetime

# create variable for current date and time, rounded to nearest second (for filename of Excel extract at end)
now = datetime.now()

now_ns = (now.strftime("%Y-%m-%d-%H-%M-%S"))
print(now_ns)

# create variable for start of reporting month and reporting month, in yyyy-mm[-dd] format 
# (start of reporting month is for SQL queries, reporting month is for filename of Excel extract)
start_of_month = datetime(2025, 6, 1)
start_of_month_d = start_of_month.strftime("%Y-%m-%d")
reporting_month = start_of_month.strftime("%Y-%m")
print(start_of_month_d)
print(reporting_month)

# create variable for CSDS month ID (1499 = February 2025, 1500 = March 2025, etc. This needs to be incremented each reporting month)
csds_month_id = 1503

2025-09-25-16-00-48
2025-06-01
2025-06


In [11]:
# Query to connect to SQL Server and run a SQL query for DQ metrics (PU5001-2)

import env
import pyodbc
import pandas as pd
import os


def data_connection_dq():
    """
    Using the user's credentials, retrieve the data from the UDAL Warehouse

    returns:
    pandas dataframe of the data
    """

    server = env.SERVER
    database = env.DATABASE
    uid = env.UID

    conn_str = (
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={uid};'
        'Authentication=ActiveDirectoryInteractive;'
    )

    conn = pyodbc.connect(conn_str)


#    table = os.getenv('table')
#   subtable = os.getenv('subtable')

    query = f"""
    DECLARE @MonthStart1 as DATE DECLARE @MonthEnd1 as DATE 
    SET @MonthStart1 = '{start_of_month_d}'
    SET @MonthEnd1 = EOMONTH(@MonthStart1)

    SELECT
	@MonthEnd1 AS [ReportingDate],
	[OrgID_Provider] as [ProviderCode],
	[CodedFinding] as [CodedFindingOrProcedure],
	count(*) as [NumberOfRecords]

	from [Reporting_MESH_CSDS].[CYP202CareActivity]
    
    where [OrgID_Provider] in ('NQA', 'NR5', 'NAY')
    and [Unique_MonthID] = '{csds_month_id}'

	group by
	[OrgID_Provider],
	[CodedFinding]
    """
    
    query2 = f"""
    DECLARE @MonthStart1 as DATE DECLARE @MonthEnd1 as DATE 
    SET @MonthStart1 = '{start_of_month_d}' 
    SET @MonthEnd1 = EOMONTH(@MonthStart1)

    SELECT
	@MonthEnd1 AS [ReportingDate],
	[OrgID_Provider] as [ProviderCode],
	[CodedProcedure] as [CodedFindingOrProcedure],
	count(*) as [NumberOfRecords]

	from [Reporting_MESH_CSDS].[CYP202CareActivity]
    
    where [OrgID_Provider] in ('NQA', 'NR5', 'NAY')
    and [Unique_MonthID] = '{csds_month_id}'

	group by
	[OrgID_Provider],
	[CodedProcedure]
    """

    # Execute the query and load it into a pandas DataFrame
    df = pd.read_sql(query, conn)
    df2 = pd.read_sql(query2, conn)
    
    # Close the connection after retrieving data
    conn.close()

    df["Metric_ID"] = 'PU5001'
    df2["Metric_ID"] = 'PU5002'

    df_combined = pd.concat([df, df2])

    return df_combined

In [12]:
# Running function created above to generate the SQL DQ query results as one merged and transformed dataframe
df = data_connection_dq()

# Creating copy of dataframe to work with in case of errors
df_copy = df

# Checking the generated dataframe looks right by inspecting first 10 rows
df_copy.head(10)

C:\TEMP\ipykernel_10204\3998403455.py:78: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\TEMP\ipykernel_10204\3998403455.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query2, conn)


,ReportingDate,ProviderCode,CodedFindingOrProcedure,NumberOfRecords,Metric_ID
0,2025-06-30,NQA,42QE.,232,PU5001
1,2025-06-30,NR5,116308004,4,PU5001
2,2025-06-30,NQA,Ua267,4,PU5001
3,2025-06-30,NQA,1B4..,94,PU5001
4,2025-06-30,NR5,162010006,925,PU5001
5,2025-06-30,NQA,Xa3Om,154,PU5001
6,2025-06-30,NR5,284196006,3,PU5001
7,2025-06-30,NR5,102572006,139,PU5001
8,2025-06-30,NR5,796571000000104,74,PU5001
9,2025-06-30,NR5,160689007,4,PU5001


In [13]:
#Function to transform dataframe into table to email to IPs

import pandas as pd

def transform_ll_dq_data(input_df):

    # Step 1: Define input data
    df = input_df

    # Step 2: Prepare numeratorValue and denominatorValue for each ProviderCode
    # numeratorValue: count of non-null values for each ProviderCode
    numerator = df[df['CodedFindingOrProcedure'].notna()].groupby(['ProviderCode', 'Metric_ID'])['NumberOfRecords'].sum().reset_index(name='numeratorValue')

    # denominatorValue: count of all rows (including nulls) for each ProviderCode
    denominator = df.groupby(['ProviderCode', 'Metric_ID'])['NumberOfRecords'].sum().reset_index(name='denominatorValue')

    # Merge numerator and denominator counts
    counts = pd.merge(numerator, denominator, on=['ProviderCode', 'Metric_ID'])

    # Step 3: Calculate value = (numeratorValue / denominatorValue) * 100
    counts['value'] = (counts['numeratorValue'] / counts['denominatorValue']) * 100

    # Step 4: Get rangeMin and rangeMax from the 'value' column
    # rangeMin = counts['value'].min()
    # rangeMax = counts['value'].max()

    # Step 5: Since the reportingDate might vary, let's associate each ProviderCode with its reportingDate(s).
    # Assuming the ReportingDate is consistent per ProviderCode, take the first ReportingDate for each ProviderCode.
    reporting_dates = df.groupby('ProviderCode')['ReportingDate'].first().reset_index()

    # Step 6: Build the final DataFrame with the specified columns and order
    df_transformed = counts.merge(reporting_dates, on='ProviderCode')

    df_transformed = df_transformed.rename(columns={
        'ProviderCode': 'providerCode',
        'ReportingDate': 'reportingDate',
        'Metric_ID': 'metricID'
    })

    # Reorder columns as requested
    df_transformed = df_transformed[
        ['metricID', 'providerCode', 'reportingDate', 'value', 
        'numeratorValue', 'denominatorValue']
    ]
    
    return df_transformed

In [14]:
# Run function above on DQ dataframe
df_copy_transformed = transform_ll_dq_data(df_copy)

# Checking the transformed dataframe looks right by inspecting first 10 rows
df_copy_transformed.head(10)

,metricID,providerCode,reportingDate,value,numeratorValue,denominatorValue
0,PU5001,NQA,2025-06-30,49.278713,303309,615497
1,PU5002,NQA,2025-06-30,32.009904,197020,615497
2,PU5001,NR5,2025-06-30,42.712913,245504,574777
3,PU5002,NR5,2025-06-30,31.893935,183319,574777


In [15]:
# Query to connect to SQL Server and run a SQL query for the LL wound code metrics (PU5003-7)

import env
import pyodbc
import pandas as pd
import os


def data_connection_ll_codes():
    """
    Using the user's credentials, retrieve the data from the UDAL Warehouse

    returns:
    pandas dataframe of the data
    """

    server = env.SERVER
    database = env.DATABASE
    uid = env.UID

    conn_str = (
        f'DRIVER={{ODBC Driver 17 for SQL Server}};'
        f'SERVER={server};'
        f'DATABASE={database};'
        f'UID={uid};'
        'Authentication=ActiveDirectoryInteractive;'
    )

    conn = pyodbc.connect(conn_str)


#    table = os.getenv('table')
#   subtable = os.getenv('subtable')

    query = f"""
        
    DECLARE @MonthStart1 as DATE
    DECLARE @MonthEnd1 as DATE
    SET @MonthStart1 = '{start_of_month_d}'
    SET @MonthEnd1 = EOMONTH(@MonthStart1)

    SELECT
    @MonthEnd1 AS [ReportingDate],
    Contacts.OrgID_Provider AS [ProviderCode],
    COUNT (DISTINCT Contacts.Person_ID) AS Unique_Patients,

    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb')
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb') THEN 'LegUlcer_Code_Present'
    ELSE 'Other' END AS Metric


    FROM [Reporting_MESH_CSDS].[CYP201CareContact] AS [Contacts]

    LEFT JOIN [Reporting_MESH_CSDS].[CYP202CareActivity] AS [Activity]
    ON Contacts.Person_ID = Activity.Person_ID
    AND Contacts.UniqueSubmissionID = Activity.UniqueSubmissionID
    AND Contacts.Unique_CareContactID = Activity.Unique_CareContactID
    AND Contacts.AuditID = Activity.AuditID

    LEFT JOIN [Reporting_MESH_CSDS].[DerLatestFlag] AS [Submissions]
    ON Contacts.UniqueSubmissionID = Submissions.UniqueSubmissionID
    AND Contacts.OrgID_Provider = Submissions.OrgID_Provider
    AND Submissions.RP_EndDate = @MonthEnd1

    WHERE 
    Contacts.Contact_Date BETWEEN @MonthStart1 AND @MonthEnd1
    AND Submissions.DerIsLatest = '1'
    AND (CASE WHEN AttendanceStatus IS NULL THEN AttendOrNot ELSE AttendanceStatus END IN ('5','6'))
    and Contacts.OrgID_Provider in ('NQA', 'NR5', 'NAY')

    GROUP BY
    Contacts.OrgID_Provider,
    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb')
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7','413167008','XaJWE','Xa6nc',
    '298006005','XaQn5','8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','Y36af','Y36b0','Ua1De','X77cy','X77cz','X79mk','P5sb.','p5vW.',
    'p5vX.','p5z9.','p5zA.','Xa6nb') THEN 'LegUlcer_Code_Present'
    ELSE 'Other' END

    union

    SELECT
    @MonthEnd1 AS [ReportingDate],
    Contacts.OrgID_Provider AS [ProviderCode],
    COUNT (DISTINCT Contacts.Person_ID) AS Unique_Patients,

    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') 
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') THEN 'LegUlcer_Present'
    WHEN Activity.CodedProcedure IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') 
    or Activity.CodedFinding IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') THEN 'LegUlcer_Assessment'
    WHEN Activity.CodedProcedure IN ('Y36af','Y36b0','Ua1De','P5sb.','p5vW.','p5vX.','p5z9.','p5zA.') or Activity.CodedFinding IN ('Y36af','Y36b0','Ua1De','P5sb.',
    'p5vW.','p5vX.','p5z9.','p5zA.') THEN 'LegUlcer_Treatment'
    WHEN Activity.CodedFinding IN ('413167008','XaJWE') or Activity.CodedProcedure IN ('413167008','XaJWE') THEN 'LegUlcer_TreatmentNotSpecified'
    WHEN Activity.CodedFinding IN ('Xa6nc','298006005','XaQn5','Xa6nb') or Activity.CodedProcedure IN ('Xa6nc','298006005','XaQn5','Xa6nb') THEN 'LegUlcer_Healed'
    ELSE 'Other' END AS Metric


    FROM [Reporting_MESH_CSDS].[CYP201CareContact] AS [Contacts]

    LEFT JOIN [Reporting_MESH_CSDS].[CYP202CareActivity] AS [Activity]
    ON Contacts.Person_ID = Activity.Person_ID
    AND Contacts.UniqueSubmissionID = Activity.UniqueSubmissionID
    AND Contacts.Unique_CareContactID = Activity.Unique_CareContactID
    AND Contacts.AuditID = Activity.AuditID

    LEFT JOIN [Reporting_MESH_CSDS].[DerLatestFlag] AS [Submissions]
    ON Contacts.UniqueSubmissionID = Submissions.UniqueSubmissionID
    AND Contacts.OrgID_Provider = Submissions.OrgID_Provider
    AND Submissions.RP_EndDate = @MonthEnd1

    WHERE 
    Contacts.Contact_Date BETWEEN @MonthStart1 AND @MonthEnd1
    AND Submissions.DerIsLatest = '1'
    AND (CASE WHEN AttendanceStatus IS NULL THEN AttendOrNot ELSE AttendanceStatus END IN ('5','6'))
    and Contacts.OrgID_Provider in ('NQA', 'NR5', 'NAY')
    
    GROUP BY
    Contacts.OrgID_Provider,
    CASE WHEN Activity.CodedFinding IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') 
    or Activity.CodedProcedure IN ('X50Bb','X50Bd','X50Be','X50Bf','402863005','238792006','238793001','828101000000106','Xa0lV','Y2bf7') THEN 'LegUlcer_Present'
    WHEN Activity.CodedProcedure IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') 
    or Activity.CodedFinding IN ('8437810000000105','446841001','XaYmM','XaX0e','XaX0f','Y1258','Y1259','XB001','X77cy','X77cz','X79mk') THEN 'LegUlcer_Assessment'
    WHEN Activity.CodedProcedure IN ('Y36af','Y36b0','Ua1De','P5sb.','p5vW.','p5vX.','p5z9.','p5zA.') or Activity.CodedFinding IN ('Y36af','Y36b0','Ua1De','P5sb.',
    'p5vW.','p5vX.','p5z9.','p5zA.') THEN 'LegUlcer_Treatment'
    WHEN Activity.CodedFinding IN ('413167008','XaJWE') or Activity.CodedProcedure IN ('413167008','XaJWE') THEN 'LegUlcer_TreatmentNotSpecified'
    WHEN Activity.CodedFinding IN ('Xa6nc','298006005','XaQn5','Xa6nb') or Activity.CodedProcedure IN ('Xa6nc','298006005','XaQn5','Xa6nb') THEN 'LegUlcer_Healed'
    ELSE 'Other' END
    """
    
    # Execute the query and load it into a pandas DataFrame
    df = pd.read_sql(query, conn)
    
    # Close the connection after retrieving data
    conn.close()

    return df

In [16]:
# Running function created above to generate the SQL DQ query results as one merged and transformed dataframe
df2 = data_connection_ll_codes()

# Creating copy of dataframe to work with in case of errors
df2_copy = df2

# Checking the generated dataframe looks right by inspecting first 10 rows
df2_copy.head(10)

C:\TEMP\ipykernel_10204\2667322711.py:136: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ReportingDate,ProviderCode,Unique_Patients,Metric
0,2025-06-30,NQA,249,LegUlcer_Present
1,2025-06-30,NQA,10386,Other
2,2025-06-30,NQA,355,LegUlcer_Assessment
3,2025-06-30,NQA,638,LegUlcer_Code_Present
4,2025-06-30,NQA,149,LegUlcer_Healed
5,2025-06-30,NQA,67,LegUlcer_Treatment
6,2025-06-30,NR5,106,LegUlcer_Present
7,2025-06-30,NR5,12544,Other
8,2025-06-30,NR5,100,LegUlcer_Assessment
9,2025-06-30,NR5,381,LegUlcer_Code_Present


In [17]:
def transform_ll_metric_data(input_df):

    # Mapping for recoding the Metric column including the new entry
    metric_map = {
        'LegUlcer_Code_Present': 'PU5003',
        'LegUlcer_Present': 'PU5004',
        'LegUlcer_Assessment': 'PU5005',
        'LegUlcer_Treatment': 'PU5006',
        'LegUlcer_Healed': 'PU5007'
    }

    # Read the Excel file into a DataFrame
    df = input_df

    # Recode Metric column and drop rows not in mapping
    df['metricID'] = df['Metric'].map(metric_map)
    df = df.dropna(subset=['metricID'])

    # Rename and reorder columns, also transfer data accordingly
    df_transformed = pd.DataFrame({
        'metricID': df['metricID'],
        'providerCode': df['ProviderCode'],
        'reportingDate': df['ReportingDate'],
        'value': df['Unique_Patients'],
        'numeratorValue': '',  # blank column
        'denominatorValue': '',# blank column
    })

    # Convert 'value' column to numeric if not already (just in case)
    df_transformed['value'] = pd.to_numeric(df_transformed['value'], errors='coerce')
 
    return df_transformed


In [18]:
# Run function above on DQ dataframe
df2_copy_transformed = transform_ll_metric_data(df2_copy)

# Checking the transformed dataframe looks right by inspecting first 10 rows
df2_copy_transformed.head(10)

,metricID,providerCode,reportingDate,value,numeratorValue,denominatorValue
0,PU5004,NQA,2025-06-30,249,,
2,PU5005,NQA,2025-06-30,355,,
3,PU5003,NQA,2025-06-30,638,,
4,PU5007,NQA,2025-06-30,149,,
5,PU5006,NQA,2025-06-30,67,,
6,PU5004,NR5,2025-06-30,106,,
8,PU5005,NR5,2025-06-30,100,,
9,PU5003,NR5,2025-06-30,381,,
10,PU5007,NR5,2025-06-30,232,,
11,PU5006,NR5,2025-06-30,11,,


In [19]:
# Concatenating the two dataframes
import pandas as pd

df_all_metrics = pd.concat([df_copy_transformed, df2_copy_transformed])

In [20]:
df_all_metrics.head(10)

,metricID,providerCode,reportingDate,value,numeratorValue,denominatorValue
0,PU5001,NQA,2025-06-30,49.278713,303309,615497
1,PU5002,NQA,2025-06-30,32.009904,197020,615497
2,PU5001,NR5,2025-06-30,42.712913,245504,574777
3,PU5002,NR5,2025-06-30,31.893935,183319,574777
0,PU5004,NQA,2025-06-30,249.000000,,
2,PU5005,NQA,2025-06-30,355.000000,,
3,PU5003,NQA,2025-06-30,638.000000,,
4,PU5007,NQA,2025-06-30,149.000000,,
5,PU5006,NQA,2025-06-30,67.000000,,
6,PU5004,NR5,2025-06-30,106.000000,,


In [21]:
# function to add metric descriptions column

def add_metric_desc(input_df):
    
    mapping = {
        'PU5001': 'Percentage completeness of the Clinical Coded Finding field in the CSDS',
        'PU5002': 'Percentage completeness of the Clinical Coded Procedure field in the CSDS',
        'PU5003': 'Number of unique patients on the community service caseload who have a lower limb code',
        'PU5004': 'Number of unique patients on the community service caseload who have a lower limb code - Presence',
        'PU5005': 'Number of unique patients on the community service caseload who have a lower limb code - Assessment',
        'PU5006': 'Number of unique patients on the community service caseload who have a lower limb code - Treatment',
        'PU5007': 'Number of unique patients on the community service caseload who have a lower limb code - Healed'
    }

    df = input_df

    df['metricDesc'] = df['metricID'].map(mapping)

    return df

In [22]:
df_all_metrics_desc = df_all_metrics

add_metric_desc(df_all_metrics_desc)

df_all_metrics_desc.head(10)

,metricID,providerCode,reportingDate,value,numeratorValue,denominatorValue,metricDesc
0,PU5001,NQA,2025-06-30,49.278713,303309,615497,Percentage completeness of the Clinical Coded ...
1,PU5002,NQA,2025-06-30,32.009904,197020,615497,Percentage completeness of the Clinical Coded ...
2,PU5001,NR5,2025-06-30,42.712913,245504,574777,Percentage completeness of the Clinical Coded ...
3,PU5002,NR5,2025-06-30,31.893935,183319,574777,Percentage completeness of the Clinical Coded ...
0,PU5004,NQA,2025-06-30,249.000000,,,Number of unique patients on the community ser...
2,PU5005,NQA,2025-06-30,355.000000,,,Number of unique patients on the community ser...
3,PU5003,NQA,2025-06-30,638.000000,,,Number of unique patients on the community ser...
4,PU5007,NQA,2025-06-30,149.000000,,,Number of unique patients on the community ser...
5,PU5006,NQA,2025-06-30,67.000000,,,Number of unique patients on the community ser...
6,PU5004,NR5,2025-06-30,106.000000,,,Number of unique patients on the community ser...


In [23]:
#function to filter dataframe by ODS code

def filter_by_prov(input_df, ods_code):
    
    df = input_df
    
    output_df = df[df['providerCode'] == ods_code]
    
    return output_df

In [24]:
df_all_metrics_desc_nqa = filter_by_prov(df_all_metrics_desc, 'NQA')

df_all_metrics_desc_nqa.head(10)

,metricID,providerCode,reportingDate,value,numeratorValue,denominatorValue,metricDesc
0,PU5001,NQA,2025-06-30,49.278713,303309,615497,Percentage completeness of the Clinical Coded ...
1,PU5002,NQA,2025-06-30,32.009904,197020,615497,Percentage completeness of the Clinical Coded ...
0,PU5004,NQA,2025-06-30,249.000000,,,Number of unique patients on the community ser...
2,PU5005,NQA,2025-06-30,355.000000,,,Number of unique patients on the community ser...
3,PU5003,NQA,2025-06-30,638.000000,,,Number of unique patients on the community ser...
4,PU5007,NQA,2025-06-30,149.000000,,,Number of unique patients on the community ser...
5,PU5006,NQA,2025-06-30,67.000000,,,Number of unique patients on the community ser...


In [25]:
df_all_metrics_desc_nr5 = filter_by_prov(df_all_metrics_desc, 'NR5')

df_all_metrics_desc_nr5.head(10)

,metricID,providerCode,reportingDate,value,numeratorValue,denominatorValue,metricDesc
2,PU5001,NR5,2025-06-30,42.712913,245504,574777,Percentage completeness of the Clinical Coded ...
3,PU5002,NR5,2025-06-30,31.893935,183319,574777,Percentage completeness of the Clinical Coded ...
6,PU5004,NR5,2025-06-30,106.000000,,,Number of unique patients on the community ser...
8,PU5005,NR5,2025-06-30,100.000000,,,Number of unique patients on the community ser...
9,PU5003,NR5,2025-06-30,381.000000,,,Number of unique patients on the community ser...
10,PU5007,NR5,2025-06-30,232.000000,,,Number of unique patients on the community ser...
11,PU5006,NR5,2025-06-30,11.000000,,,Number of unique patients on the community ser...


In [26]:
"""
df_all_metrics_desc_nay = df_all_metrics_desc

filter_by_prov(df_all_metrics_desc_nqa, 'NAY')

df_all_metrics_desc_nay.head(10)
"""

"\ndf_all_metrics_desc_nay = df_all_metrics_desc\n\nfilter_by_prov(df_all_metrics_desc_nqa, 'NAY')\n\ndf_all_metrics_desc_nay.head(10)\n"

In [27]:
# Function to export dataframe to Excel

def export_to_excel(df, output_file):

    df.to_excel(output_file, sheet_name='Insert', index=False)

    print(f"Transformation complete. Data saved to {output_file}")
    

In [28]:
export_to_excel(df_all_metrics_desc_nqa, f'C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nqa_all_metrics_{reporting_month}_{now_ns}.xlsx')

Transformation complete. Data saved to C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nqa_all_metrics_2025-06_2025-09-25-16-00-48.xlsx


In [31]:
export_to_excel(df_all_metrics_desc_nr5, f'C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nr5_all_metrics_{reporting_month}_{now_ns}.xlsx')

Transformation complete. Data saved to C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nr5_all_metrics_2025-06_2025-09-25-16-00-48.xlsx


In [30]:
# export_to_excel(df_all_metrics_desc_nay, f'C:/Users/owen.kenn (DDCC7590)/NHS/Quality and Improvement Analytics - Productivity/Wound Care/Wound Care - LL 2025/Excel outputs/nay_all_metrics_{reporting_month}_{now_ns}.xlsx')